In [3]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress

In [4]:
# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 1. STEP 1: FETCH UNIVERSE
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        # Filter out Test issues and ETFs
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        # Filter for length < 5 to avoid warrants/rights generally
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")
        
    return list(set(tickers))

# ==========================================
# 2. STEP 2: FINANCIALS ONLY FILTER
# ==========================================
def filter_financials_universe(ticker_list):
    print(f"\n--- STEP 2: Filtering for Financial Services (YahooQuery) ---")
    
    # Configuration
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 1_000_000
    MIN_CAP_CA = 100_000_000
    MIN_VOL_CA = 100_000
    
    valid_candidates = []
    chunk_size = 500 # Safe size
    
    cnt_us = 0
    cnt_ca = 0
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # We need 'summaryProfile' for the Sector
            yq = Ticker(chunk, asynchronous=False)
            data = yq.get_modules("summaryProfile summaryDetail price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # 1. Check Sector (Must be Financial)
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                if 'Financial' not in sector and 'Real Estate' not in sector:
                    continue

                # 2. Extract Basic Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                
                fin_mod = data[symbol].get('financialData', {})
                
                # 3. Extract Rating
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key and isinstance(rec_key, str):
                    rec_key = rec_key.lower().strip()
                
                # 4. Filters
                if curr_price < MIN_PRICE: continue
                
                # Rating Check
                if rec_key not in ['buy', 'strong_buy', 'strong buy']: continue

                is_canada = symbol.endswith('.TO')
                if is_canada:
                    if mkt_cap < MIN_CAP_CA: continue
                    if avg_vol < MIN_VOL_CA: continue
                else:
                    if mkt_cap < MIN_CAP_US: continue
                    if avg_vol < MIN_VOL_US: continue

                # Add to list
                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Sector': sector,
                    'Rating': rec_key
                })
                
                if is_canada: cnt_ca += 1
                else: cnt_us += 1
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

# ==========================================
# 3. SPECIALIZED FINANCIALS ENGINE
# ==========================================
def run_financials_analysis(df):
    """
    Takes the raw list of Financial stocks (Banks/Insurance),
    fetches valuation metrics, and filters for Value & Income.
    """
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return

    print(f"\n--- STEP 3: ANALYZING {len(df)} FINANCIAL STOCKS ---")
    print("Fetching P/E, P/B, Dividend, and Analyst Ratings...\n")
    
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        
        # Progress bar
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(df)}...", end='\r')
        
        try:
            # Fetch Data
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 1. VALUATION METRICS
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            
            # 2. EFFICIENCY & INCOME
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            # 3. ANALYST RATING
            recom = info.get('recommendationMean', None)
            if recom is None: 
                recom = 3.0 # Fallback

            # 4. GROWTH
            rev_growth = info.get('revenueGrowth', np.nan)

            # Save it
            bank_data.append({
                'Ticker': ticker,
                'Company': row.get('Company', ticker),
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield * 100, 2),
                'Recom': float(recom) if recom is not None else 3.0,
                'Rev_Growth': rev_growth,
                'Sector': row.get('Sector', 'Financial')
            })
            
        except Exception:
            continue
            
    # Create the Enriched DataFrame
    rich_fins_df = pd.DataFrame(bank_data)
    
    if rich_fins_df.empty:
        print("Could not fetch details for financial stocks.")
        return

    # --- FIX: DATA CLEANING BLOCK ---
    # Force columns to numeric. Any strings (like 'N/A') become NaN.
    cols_to_clean = ['P/E', 'P/B', 'ROE', 'Yield%', 'Recom']
    for col in cols_to_clean:
        rich_fins_df[col] = pd.to_numeric(rich_fins_df[col], errors='coerce')
    # --------------------------------

    # ==========================================
    # FILTER 1: "UNDERVALUED BANKING" (The Value Play)
    # Criteria: Cheap (P/E < 15), Trading near assets (P/B < 1.2), Profitable (ROE > 8%)
    # ==========================================
    value_picks = rich_fins_df[
        (rich_fins_df['P/E'] < 15) & 
        (rich_fins_df['P/B'] < 1.2) & 
        (rich_fins_df['ROE'] > 0.08)
    ].copy()
    
    print(f"\n\n--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---")
    if not value_picks.empty:
        # Sort by Price-to-Book (Cheapest assets first)
        display_cols = ['Ticker', 'Price', 'P/B', 'P/E', 'ROE', 'Yield%', 'Recom']
        try:
            display(value_picks.sort_values(by='P/B', ascending=True)[display_cols])
        except:
            print(value_picks.sort_values(by='P/B', ascending=True)[display_cols])
    else:
        print("No stocks met the strict value criteria.")

    # ==========================================
    # FILTER 2: "INCOME COMPOUNDERS" (The Dividend Play)
    # Criteria: High Yield (>2.5%), Sustainable P/E (<20), Buy Rating (< 2.5)
    # ==========================================
    income_picks = rich_fins_df[
        (rich_fins_df['Yield%'] >= 2.5) & 
        (rich_fins_df['P/E'] < 20) &
        (rich_fins_df['ROE'] > 0.05) &
        (rich_fins_df['Recom'] <= 2.5) 
    ].copy()

    print(f"\n--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---")
    if not income_picks.empty:
        # Sort by Yield (Highest income first)
        display_cols = ['Ticker', 'Price', 'Yield%', 'P/E', 'Recom', 'Sector']
        try:
            display(income_picks.sort_values(by='Yield%', ascending=False)[display_cols])
        except:
            print(income_picks.sort_values(by='Yield%', ascending=False)[display_cols])

    return rich_fins_df

# --- EXECUTION ---
raw_tickers = get_raw_universe()
financial_df = filter_financials_universe(raw_tickers)
if not financial_df.empty:
    run_financials_analysis(financial_df)

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Filtering for Financial Services (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 174

--- STEP 3: ANALYZING 174 FINANCIAL STOCKS ---
Fetching P/E, P/B, Dividend, and Analyst Ratings...

   Analyzing 170/174...

--- FILTER 1: UNDERVALUED BANKS (P/E < 15 & P/B < 1.2) ---


,Ticker,Price,P/B,P/E,ROE,Yield%,Recom
95,MFG,7.320,0.257894,14.076924,0.09280,268.0,2.50000
55,MARA,9.590,0.703853,3.731518,0.23066,0.0,1.92308
116,BMNR,28.310,0.764391,2.114264,0.08016,4.0,1.00000
5,OBDC,12.730,0.854764,9.028369,0.09772,888.0,1.58333
79,RITM,11.160,0.870175,7.643835,0.11037,896.0,1.44444
122,KMPR,40.920,0.901679,10.546391,0.08678,313.0,2.50000
19,OZK,47.160,0.923041,7.606452,0.12351,382.0,2.50000
12,EFC,13.840,1.023820,10.484848,0.09283,1127.0,1.75000
51,VICI,28.130,1.086436,10.695817,0.10357,627.0,1.66667
81,DX,13.880,1.093430,7.886363,0.11922,1470.0,1.83333



--- FILTER 2: INCOME GENERATORS (Yield > 2.5% & Buy Rating) ---


,Ticker,Price,Yield%,P/E,Recom,Sector
81,DX,13.880,1470.0,7.886363,1.83333,Real Estate
48,PFLT,9.150,1344.0,12.708332,2.12500,Financial Services
3,AGNC,10.850,1327.0,16.194030,2.14286,Real Estate
31,NLY,23.140,1210.0,10.238937,2.07692,Real Estate
12,EFC,13.840,1127.0,10.484848,1.75000,Real Estate
...,...,...,...,...,...,...
87,XP,16.440,109.0,9.670588,1.75000,Financial Services
77,WT,12.400,97.0,19.375000,1.66667,Financial Services
23,INTR,8.450,95.0,16.900000,2.00000,Financial Services
114,PNFP,100.795,95.0,12.856504,1.93333,Financial Services


In [ ]:
import os
import time

# ==========================================
# 4. SEPARATE SAVE FUNCTION
# ==========================================
def save_to_excel_separate(df):
    """
    Saves the 'final_results_df' to OneDrive.
    """
    if df is None or df.empty:
        print("❌ No data found. Did you run the main analysis cell first?")
        return

    # 1. Setup Filename
    today_date = time.strftime("%Y-%m-%d")
    file_nickname = f"Financial_Stock_Picks_{today_date}.xlsx"
    
    # 2. Setup Folder (Explicit Path)
    onedrive_folder = r"C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks"
    full_path = os.path.join(onedrive_folder, file_nickname)

    print(f"Saving to: {full_path}...")

    # 3. Filter the Lists again for separate sheets
    value_picks = df[df['Strategy'] == 'Value (Undervalued)'].sort_values(by='P/B', ascending=True)
    income_picks = df[df['Strategy'] == 'Income (Dividend)'].sort_values(by='Yield%', ascending=False)

    try:
        if not os.path.exists(onedrive_folder):
            os.makedirs(onedrive_folder, exist_ok=True)

        with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
            value_picks.to_excel(writer, sheet_name='Value Picks', index=False)
            income_picks.to_excel(writer, sheet_name='Income Picks', index=False)
            df.to_excel(writer, sheet_name='All Financials', index=False)
            
        print(f"✅ Success! Excel file created.")
        print(f"   - Value Picks: {len(value_picks)}")
        print(f"   - Income Picks: {len(income_picks)}")
        
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Close the Excel file if it is currently open.")

# --- TRIGGER SAVE ---
# This checks if the dataframe from Part 1 exists before running
if 'final_results_df' in locals():
    save_to_excel_separate(final_results_df)
else:
    print("⚠️ 'final_results_df' is missing. Please run the analysis code block first.")

Please run the analysis step first.


In [6]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 11 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...


C:\Users\James\AppData\Local\Temp\ipykernel_66900\4051987151.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)



--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.49,-1.10,17.68,-46.32,1.40,21.75,0.67,3.18
0,ADMA,19.28,-1.33,17.90,-24.89,1.00,30.00,0.41,3.26
9,SEI,44.65,-0.93,32.46,-21.72,1.17,65.45,0.31,5.90
7,MIR,23.75,-0.92,19.89,-21.56,1.12,30.62,0.34,3.49
10,UBER,81.26,0.14,84.45,-20.33,1.47,112.40,0.33,2.41
5,FLEX,63.33,-0.35,48.21,-12.31,1.50,76.00,0.23,2.90
2,ARCC,20.20,1.00,20.48,-9.77,1.27,22.64,0.85,1.38
8,ONB,22.98,-0.35,21.40,-3.76,1.85,25.92,0.70,2.14
1,APG,39.57,0.43,31.34,-2.49,1.45,43.40,0.29,1.92
3,BANC,19.70,0.15,15.35,-1.84,1.27,22.14,0.44,2.18
